In [ ]:
# Import Dependencies
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
#Here I am declaring a base using the automap_base function
Base = automap_base()

# Now that I have my variable base defined I can use it to reflect the database tables in
Base.prepare(engine, reflect=True)

In [ ]:
# Print all of the tables mapped to the Base
Base.classes.keys()

In [ ]:
# Now I want a list of the columns in each table
inspector = inspect(engine)

In [ ]:
# First I am going to get a list of columns from measurment
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# First I am going to get a list of columns from measurment
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(measurement).order_by(measurement.date.desc()).first()
#This is a string I want to convert it to a date
most_rec_date = datetime.strptime(most_recent_date.date, '%Y-%m-%d')
print(most_rec_date)

In [ ]:
# Find one year ago from this date
year_ago_most_rec_date = most_rec_date - relativedelta(years=1)
print(year_ago_most_rec_date)

In [ ]:
# I have to go back an extra day or else the last date in a full year back is not included
year_ago_most_rec_date = year_ago_most_rec_date - relativedelta(days=1)

In [ ]:
#Query the last 12 months of the measurement data
last_year_prcp= session.query(measurement.date, measurement.prcp).filter(measurement.date>=year_ago_most_rec_date).\
                filter(measurement.date<most_rec_date).all()

In [ ]:
# Save the query results as a Pandas DataFrame
last_year_df = pd.DataFrame(last_year_prcp, columns=['date', 'prcp'])
last_year_df.set_index('date', inplace=True, )
last_year_df.head(5)

In [ ]:
# Here I am going to sort by date to make sure the dataframe is in the correct order 
last_year_df = last_year_df.sort_values("date")
last_year_df.head(10)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
last_year_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
distinct_stations = session.query(measurement).group_by(measurement.station).count()
print(distinct_stations)

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
session.query(measurement.station, func.count(measurement.station)).group_by(measurement.station).\
    order_by(func.count(measurement.station).desc()).all()

In [ ]:
# We can see the which station had the most rows by looking at the output above but here I will ust code to automatically find out 
most_active_station = session.query(measurement.station).group_by(measurement.station).\
                        order_by(func.count(measurement.station).desc()).first()
print(most_active_station.station)

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
max_temp = session.query(measurement.tobs, func.max(measurement.tobs)).filter(measurement.station == most_active_station.station).one()
min_temp = session.query(measurement.tobs, func.min(measurement.tobs)).filter(measurement.station == most_active_station.station).one()
mean_temp = session.query(func.avg(measurement.tobs)).filter(measurement.station == most_active_station.station).one()
print(f"The most common station in the table is {most_active_station.station}")
print(f"The maximum temperature for this station is {max_temp.tobs}")
print(f"The minimum temperature for this station is {min_temp.tobs}")
print(f"The average temperature for this station is {mean_temp}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station
last_year_tobs = session.query(measurement.tobs).filter(measurement.station == most_active_station.station).\
    filter(measurement.date>=year_ago_most_rec_date).filter(measurement.date<most_rec_date).all()

In [ ]:
# Clean up query results to get a list of numbers
last_year_temp = []
for i in range(0,len(last_year_tobs)):
    last_year_temp.append(last_year_tobs[i].tobs)
print(last_year_temp)

In [ ]:
# Plot histogram of 
plt.hist(last_year_temp,bins=12)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()

# Close session

In [ ]:
# Close Session
session.close()